In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv


In [1]:
import numpy as np
import pandas as pd
import itertools
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb

In [3]:
train = pd.read_csv('/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv')

In [4]:
print(train.head)

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                             

In [6]:
print('\n \nNumber of Null values in Train Set: ', train['tweet'].isna().sum())


 
Number of Null values in Train Set:  0


In [7]:
# Secluding labels in a new pandas dataframe for supervised learning
train_labels = train['class']# Splitting data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train['tweet'], train_labels, test_size=0.1, random_state=0)

In [8]:
print(x_train.head)

<bound method NDFrame.head of 18916    RT @dreamthievin: Do I get to pick the actress...
8413     Charles was eatin trash ass Dominoes in that p...
24290    nobody want a nigga to take them out on an exp...
12653    Lookin at u hoes #federalboobieinspector http:...
11879    It ain't nunn to cut that bitch off&#9986;&#65...
                               ...                        
13123    My partna from Mississippi told me 2weeks ago,...
19648    RT @mckayllaa: I wish I had pretty colored eyes .
9845     Hoes all on my bandwagon, your bitch gaggin, s...
10799    I never played a bitch... But bitches most def...
2732                @CSNChicago Punk ass bitch. #BullsTalk
Name: tweet, Length: 22304, dtype: object>


In [9]:
print(train.sample(10))

       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
6670         6856      3            0                   3        0      1   
18832       19249      3            0                   1        2      2   
12134       12443      3            0                   0        3      2   
15506       15870      3            0                   3        0      1   
18172       18577      3            0                   3        0      1   
10069       10342      3            1                   2        0      1   
4506         4638      3            0                   3        0      1   
1652         1687      3            0                   3        0      1   
17820       18222      3            2                   1        0      0   
8367         8597      3            0                   3        0      1   

                                                   tweet  
6670   @metroadlib I be THIIIIIIIS close to going rea...  
18832  RT @daliasza: &#8220;@NoChi

In [10]:
train.describe()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


In [11]:
train.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [12]:
# Secluding labels in a new pandas dataframe for supervised learning
train_labels = train['class']

In [13]:
# Splitting data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train['tweet'], train_labels, test_size=0.3, random_state=0)

In [14]:
x_train.describe()

count                                                 17348
unique                                                17348
top       none here Chantel lol RT "@TellyJellyBelly: Th...
freq                                                      1
Name: tweet, dtype: object

In [15]:
x_train.head

<bound method NDFrame.head of 24291    none here Chantel lol RT "@TellyJellyBelly: Th...
3078     @Dswizzle3 @Dswizzle boy u look like big bird ...
16565    RT @MyDickNeedsCPR: What lonely hoe made this?...
6425     @kieffer_jason am going to hurt more then your...
8728     Did you all hear this #bundy redneck? Someone ...
                               ...                        
13123    My partna from Mississippi told me 2weeks ago,...
19648    RT @mckayllaa: I wish I had pretty colored eyes .
9845     Hoes all on my bandwagon, your bitch gaggin, s...
10799    I never played a bitch... But bitches most def...
2732                @CSNChicago Punk ass bitch. #BullsTalk
Name: tweet, Length: 17348, dtype: object>

In [16]:
print(x_train.sample(20))

20016    RT @sidneeyrosee: fuckin skyler white is such ...
20809    Shout out to you ungrateful, inconsiderate bit...
15756    RT @JareerKassis: If there was an invisible ce...
17228    RT @SeanTheTerrible: I don't care how pretty y...
3466                         @I_GotOne he had the hoes too
14550    RT @CHAMPAGNE_____: karma's a bitch. don't dis...
3587     @J_Nastyy_ @Bum_Farto why would a white man ev...
10728    I love this ho @nerissaloren http://t.co/I5obSnjj
9942         How you like them apples you pineapple bitch?
7842      At the MLK march tryna find a man bitch we see U
17409    RT @StayTrippyyyy_: I love bad bitches, that's...
16612    RT @NaziaBee: &#8220;@1waynostra: So jus don't...
18816    RT @curlyhairkilla: You gotta be a really weak...
12258               Known to take a nigga bitch real quick
23586            apparently I'm pathetic trash alright (-:
20886    Sloppy ignorance in all my pockets &#128176;&#...
6780     @normingtonTJ @hollywood_ferg7 lmfao sorry bit.

In [17]:
print(y_train.sample(20))

14261    1
15346    2
11197    1
11050    1
17389    1
9101     1
16253    1
17001    1
4322     1
5387     1
9244     1
590      1
20637    1
24096    2
1326     1
16720    1
490      1
2129     2
15561    1
4474     1
Name: class, dtype: int64


In [19]:
# Setting up Term Frequency - Inverse Document Frequency Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7)# Fit and transform training set and transform test set
tfidf_train = tfidf.fit_transform(x_train) 
tfidf_test = tfidf.transform(x_test)

In [20]:
print(tfidf_train)

  (0, 3635)	0.2705819994100818
  (0, 8318)	0.48776898199497065
  (0, 26605)	0.3391317346008009
  (0, 11560)	0.16446926652676525
  (0, 10569)	0.19288442164195874
  (0, 24130)	0.48776898199497065
  (0, 21048)	0.10795976881629277
  (0, 15194)	0.20220421800912258
  (0, 5257)	0.4681255081508819
  (1, 23372)	0.34014800690217456
  (1, 21744)	0.4714055407636158
  (1, 3734)	0.24440946768480865
  (1, 14919)	0.15319423382431546
  (1, 15228)	0.2333413908679433
  (1, 4199)	0.2774177643985974
  (1, 8076)	0.4714055407636158
  (1, 8077)	0.396049245965409
  (1, 3635)	0.2615046435120083
  (2, 8344)	0.5891593143042168
  (2, 11819)	0.1861490376567077
  (2, 11551)	0.2118811693860035
  (2, 15209)	0.45742007134652823
  (2, 17033)	0.5891593143042168
  (2, 21048)	0.13040087770260192
  (3, 9156)	0.5287191483322847
  :	:
  (17343, 11560)	0.11795391427689225
  (17343, 10569)	0.13833266856577753
  (17344, 16025)	0.6145078572654185
  (17344, 27112)	0.3871106210054667
  (17344, 19437)	0.38354031228552604
  (17344, 8

In [ ]:
param_grid = {'n_neighbors':np.arange(1,50), 'weights':['uniform','distance'], 'leaf_size':np.arange(1,10)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(tfidf_train, y_train)
y_pred = knn_cv.predict(tfidf_test)
print(knn_cv.best_params_)
print(knn_cv.best_score_)
print(classification_report(y_pred,y_test))

In [ ]:
# Setting up Passive Aggressive Classifier
pac = PassiveAggressiveClassifier(max_iter = 50)# Fitting on the training set
pac.fit(tfidf_train, y_train)# Predicting on the test set
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')

In [ ]:
# Creating confusion matrix with columns as True Positive, False Negative, False Positive and True Negative 
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1)
sn.heatmap(df_cm, annot=True, annot_kws={'size':14}, fmt='d').set_title('Confusion Matrix')
plt.show()# Creating classification report
print('\nClassification Report: \n', classification_report(y_test, (y_pred > 0.5)))

In [ ]:
params_rf = {'n_estimators':[100,200,300,400,500],
            'max_depth':[4,6,8,10,12,14],
            'max_features':['log2','sqrt']}
rf = RandomForestClassifier()
grid_rf = GridSearchCV(estimator = rf,
                      param_grid = params_rf,
                      cv=3,
                      scoring = 'neg_mean_squared_error',
                      verbose = 1,
                      n_jobs = -1)
grid_rf.fit(tfidf_train, y_train)
y_pred = grid_rf.predict(tfidf_test)
print(grid_rf.best_params_)
print(grid_rf.best_score_)
print(classification_report(y_pred,y_test))

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(tfidf_train,y_train)
y_pred = model.predict(tfidf_test)
#print(model.best_params_)
#print(model.best_score_)
print(classification_report(y_pred,y_test))

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(tfidf_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(tfidf_test)
print(classification_report(y_pred,y_test))

In [ ]:
# compare soft voting ensemble to standalone classifiers
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=2)
	return X, y

# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('svm1', SVC(probability=True, kernel='poly', degree=1)))
	models.append(('svm2', SVC(probability=True, kernel='poly', degree=2)))
	models.append(('svm3', SVC(probability=True, kernel='poly', degree=3)))
	models.append(('svm4', SVC(probability=True, kernel='poly', degree=4)))
	models.append(('svm5', SVC(probability=True, kernel='poly', degree=5)))
	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='soft')
	return ensemble

# get a list of models to evaluate
def get_models():
	models = dict()
	models['svm1'] = SVC(probability=True, kernel='poly', degree=1)
	models['svm2'] = SVC(probability=True, kernel='poly', degree=2)
	models['svm3'] = SVC(probability=True, kernel='poly', degree=3)
	models['svm4'] = SVC(probability=True, kernel='poly', degree=4)
	models['svm5'] = SVC(probability=True, kernel='poly', degree=5)
	models['soft_voting'] = get_voting()
	return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# define dataset
#X, y = get_dataset()
# get the models to evaluate

y =train_labels 

#tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7)# Fit and transform training set and transform test set
X = tfidf.fit_transform(train['tweet']) 
#tfidf_test = tfidf.transform(x_test)                                                    

models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('svm1', SVC(kernel='poly', degree=1)))
	models.append(('svm2', SVC(kernel='linear')))
	#models.append(('svm3', SVC(kernel='poly', degree=3)))
	models.append(('svm4', SVC( kernel='rbf')))
	#models.append(('svm5', SVC(kernel='poly', degree=5)))
	models.append(('knn', KNeighborsClassifier(leaf_size = 1, n_neighbors = 6, weights= 'uniform')))
	models.append(('LR', LogisticRegression(solver ='lbfgs', multi_class ='multinomial', max_iter = 200)))
	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='hard')
	return ensemble

# get a list of models to evaluate
def get_models():
	models = dict()
	models['svm1'] = SVC(kernel='poly', degree=1)
	models['svm2'] = SVC(kernel='linear')
	#models['svm3'] = SVC(kernel='poly', degree=3)
	models['svm4'] = SVC(kernel='rbf')
	#models['svm5'] = SVC(kernel='poly', degree=5)
	models['knn'] = KNeighborsClassifier(leaf_size = 1, n_neighbors = 6, weights= 'uniform')
	models['LR'] = LogisticRegression(solver ='lbfgs', multi_class ='multinomial', max_iter = 200)
	models['hard_voting'] = get_voting()
	return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# define dataset
#X, y = get_dataset()
# get the models to evaluate

y =train_labels 

#tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7)# Fit and transform training set and transform test set
X = tfidf.fit_transform(train['tweet']) 
#tfidf_test = tfidf.transform(x_test)                                                    

models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	model.fit(tfidf_train, y_train)
	y_pred = model.predict(tfidf_test)
	print('>%s' % (name))
	print(classification_report(y_pred,y_test))
	names.append(name)

# plot model performance for comparison
pyplot.boxplot(result, labels=names, showmeans=True)
pyplot.show()